In [2]:
!nvidia-smi

Fri May 20 23:42:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -q numpy requests nlpaug
!pip install -q transformers sentencepiece
!pip install -q nltk
!pip install -q sacremoses

     |████████████████████████████████| 410 kB 8.2 MB/s 
     |████████████████████████████████| 4.2 MB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 59.6 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 6.6 MB 56.8 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 880 kB 9.8 MB/s 


In [4]:
device = "cuda"

In [5]:
import nlpaug.augmenter.word as naw
insert_aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert", device = device)
sub_aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", device = device)
syn_aug = naw.SynonymAug(aug_src='wordnet')
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en',
    device = device
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
def insert(text):
    augmented_text = insert_aug.augment(text)
    return augmented_text

In [8]:
def substitute(text):
    augmented_text = sub_aug.augment(text)
    return augmented_text

In [9]:
def back_translation(text):
    augmented_text = back_translation_aug.augment(text)
    return augmented_text

In [10]:
def synonym(text):
    augmented_text = syn_aug.augment(text)
    return augmented_text

In [11]:
ls_aug = [synonym, insert, substitute, back_translation]

In [12]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

df = pd.read_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_train.csv', header=0)
df.head()

,text_clean,Sentiment
0,it s a confusing odd time for the shopping pub...,Negative
1,in 2019 d2c ecommerce sales reached 1428 billi...,Positive
2,chinese residents are paying exorbitant prices...,Negative
3,list of supermarkets grocery shops and vegetab...,Neutral
4,its there any wonder tesco and other supermark...,Positive


In [13]:
df_neutral = df[df['Sentiment'] == "Neutral"]
thresh = [0.15, 0.15, 0.15, 0.5]

for text in tqdm(df_neutral['text_clean']):
    sample = np.random.uniform(size=4) < thresh
    for i in range(4):
        if sample[i]:
            text = ls_aug[i](text)
    new_row = {"text_clean": text, "Sentiment": "Neutral"}
    df = df.append(new_row, ignore_index=True)

  0%|          | 0/7560 [00:00<?, ?it/s]

In [14]:
df = df.sample(frac = 1)
df["Sentiment"].value_counts()

Neutral               15120
Positive              11381
Negative               9889
Extremely Positive     6618
Extremely Negative     5475
Name: Sentiment, dtype: int64

In [15]:
df.to_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/aug_clean_train.csv', header=True, index=False)